In [173]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder

from collections import Counter

In [174]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [175]:
random_state = 0

In [176]:
df = pd.read_csv('dataset_diabetes/diabetic_data_train.csv')

In [177]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

In [178]:
# Remove 'encounter_id', 'patient_nbr', 'weight',  'payer_code', 'medical_specialty' columns
df = df[[col for col in df.columns if col not in ['encounter_id', 'patient_nbr', 'weight', 'payer_code', 'medical_specialty']]]

In [179]:
df.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

# Creating data label

# How to handle features

OHE = One hot encoding

+ PREPROCESSING+OHE: 'admission_type_id', 'admission_source_id', 'diag_1', 'diag_2', 'diag_3'


+ PREPROCESSING+NUMERICAL: 'age', 'discharge_disposition_id', 'change', 'diabetesMed'


+ OHE (using categorical features as is): 'race', 'gender', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone','metformin-pioglitazone'


+ NO PREPROCESSING NEEDED (numerical): 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses'

       
+ NOT USING (at all, due to not being helpful or missing data): 'encounter_id', 'patient_nbr', 'weight',  'payer_code', 'medical_specialty'

In [180]:
[(key, val/len(df.readmitted)) for (key, val) in Counter(df.readmitted).most_common()]

[('NO', 0.5393799439886012),
 ('>30', 0.3491868520611212),
 ('<30', 0.1114332039502776)]

In [181]:
def binarize_readmitted(x):
    if x in ['>30', '<30']: # readmitted in less than 30 days is positive class
        return 1 
    else:
        return 0 

In [182]:
y = df.apply(lambda x: binarize_readmitted(x['readmitted']), axis=1).values

In [184]:
[(key, val/len(df.readmitted)) for (key, val) in Counter(y).most_common()]

[(0, 0.5393799439886012), (1, 0.4606200560113988)]

# Preprocessing Features

Mapping age groups to single numerical value (age in the middle)

### Age

In [185]:
def map_age(x):
    if x == '[0-10)':
        return 5
    elif x == '[10-20)':
        return 15
    elif x == '[20-30)':
        return 25
    elif x == '[30-40)':
        return 35
    elif x == '[40-50)':
        return 45
    elif x == '[50-60)':
        return 55
    elif x == '[60-70)':
        return 65
    elif x == '[70-80)':
        return 75
    elif x == '[80-90)':
        return 85
    else:
        return 95

In [186]:
df['age_processed'] = df.apply(lambda x: map_age(x['age']), axis=1)

### Admission Type

Combine Not Available vs. NULL vs. Not Mapped into one and leave the remaining as is

In [187]:
admission_dict = {1: 'Emergency', 2: 'Urgent', 3: 'Elective', 4: 'Newborn', 7: 'Trauma Center'}
def map_admission(x):
    if x in [5,6,8]:
        return 'Not Available'
    else:
        return admission_dict[x]

In [188]:
df['admission_type_processed'] = df.apply(lambda x: map_admission(x['admission_type_id']), axis=1)

### Discharge 

Only use discharge home, since it is the dominant value. Discharge to home is coded as 1.

In [189]:
def discharged_home(x):
    if x == 1:
        return 1 # Discharged home
    else:
        return 0 # Other 

In [190]:
df['discharged_processed'] = df.apply(lambda x: discharged_home(x['discharge_disposition_id']), axis=1)

### Source
Only using emergency room, referral (physician, clinical, or HMO), or other (everything else)

In [191]:
def map_source(x):
    if x in [1,2,3]:
        return 'Referral' 
    elif x == 7:
        return 'Emergency Room'
    else:
        return 'Other'

In [192]:
df['source_processed'] = df.apply(lambda x: map_source(x['admission_source_id']), axis=1)

### Diag1, Diag2, Diag3

In [193]:
def map_diag(x):
    if x[0].isdigit() == False:
        return 'other'
    
    float_x = float(x)
    if (float_x >= 390 and float_x <= 459) or float_x == 785: # 390–459, 785
        return 'circulatory'
    elif (float_x >= 460 and float_x <= 519) or float_x == 786: # 460–519, 786
        return 'respiratory'
    elif (float_x >= 520 and float_x <= 579) or float_x == 787: # 520–579, 787
        return 'digestive'
    elif float_x >= 250 and float_x < 251: # 250.xx
        return 'diabetes'
    elif float_x >= 800 and float_x <= 999: # 800–999
        return 'injury'
    elif float_x >= 710 and float_x <= 739: # 710–739
        return 'musculoskeletal'
    elif (float_x >= 580 and float_x <= 629) or float_x == 788: # 580–629, 788
        return 'genitourinary'
    elif float_x >= 140 and float_x <= 239: # 140–239
        return 'neoplasms'
    else:
        return 'other'

In [194]:
df['diag1_processed'] = df.apply(lambda x: map_diag(x['diag_1']), axis=1)
df['diag2_processed'] = df.apply(lambda x: map_diag(x['diag_2']), axis=1)
df['diag3_processed'] = df.apply(lambda x: map_diag(x['diag_3']), axis=1)

### Change
Making change in prescribed diabetic medication binary (0 or 1) instead of 'Ch'/'No'

In [195]:
def binarize_yn(x):
    if x in ['Yes', 'Ch']: 
        return 1 
    else:
        return 0 

In [196]:
df['change_processed'] = df.apply(lambda x: binarize_yn(x['change']), axis=1)

### diabetesMed

In [197]:
df['diabetesMed_processed'] = df.apply(lambda x: binarize_yn(x['diabetesMed']), axis=1)

# One hot Encoding Features

Resources on one hot encoding:
- https://kiwidamien.github.io/are-you-getting-burned-by-one-hot-encoding.html
- https://stackoverflow.com/questions/44601533/how-to-use-onehotencoder-for-multiple-columns-and-automatically-drop-first-dummy/44601764

In [199]:
ohe = OneHotEncoder(categories='auto')

In [200]:
df_sub = df[['admission_type_processed', 'source_processed',
            'diag1_processed', 'diag2_processed', 'diag3_processed', 'race', 'gender', 
            'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 
            'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 
            'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 
            'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 
            'glimepiride-pioglitazone', 'metformin-rosiglitazone','metformin-pioglitazone']]

In [201]:
feature_arr = ohe.fit_transform(df_sub).toarray()

In [203]:
feature_labels_ = ohe.categories_

In [204]:
feature_labels = []
for col, values in zip(df_sub.columns, feature_labels_):
    for val in values:
        feature_labels.append(col+'_'+val)

In [205]:
df_ohe = pd.DataFrame(feature_arr, columns = feature_labels)

In [206]:
df_ohe.shape

(81412, 124)

In [207]:
df_num = df[['age_processed', 'discharged_processed', 'change_processed', 'diabetesMed_processed', 'time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses']]

In [208]:
df_num.shape

(81412, 12)

# Training Model

In [265]:
X = pd.concat([df_ohe, df_num], axis=1)

In [266]:
print(X.shape, y.shape)

(81412, 136) (81412,)


In [267]:
skf = StratifiedKFold(n_splits=5, random_state=0)

In [268]:
accuracy_training = []
accuracy_cv = []
auroc_training = []
auroc_cv = []

In [269]:
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = RandomForestClassifier(max_depth=10, random_state=0)
    clf.fit(X_train, y_train)
    y_pred_train = clf.predict(X_train)
    y_pred_cv = clf.predict(X_test)
    accuracy_training.append(accuracy_score(y_train, y_pred_train))
    auroc_training.append(roc_auc_score(y_train, y_pred_train))
    accuracy_cv.append(accuracy_score(y_test, y_pred_cv))
    auroc_cv.append(roc_auc_score(y_test, y_pred_cv))

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The de

In [270]:
print(np.average(accuracy_training), np.std(accuracy_training))
print(np.average(accuracy_cv), np.std(accuracy_cv))

0.8507713841683817 0.002520721146944525
0.610806733164251 0.0036864898230898504


In [271]:
print(np.average(auroc_training), np.std(auroc_training))
print(np.average(auroc_cv), np.std(auroc_cv))

0.8454788260730892 0.0027661090505620655
0.6026641556631278 0.0037792351273024955


Notes: 

+ Both train and cross val performance is very low for max depth between 5-10 -> which indicates underfitting to the data/high bias in the model. Increasing max_depth increases training performance dramatically but does not impact cross val performance significantly. 
+ Tried only using the numerical features and get roughly the same performance. 
+ How to deal with dropping first column/redundancy in one hot encoding?
+ Tree based models are fine without rescaling (which is good because I did not rescale) but are not the best with sparse data -> are results different for other models?